# 实时获取上海证券交易所市场总览数据

- URL: <https://www.sse.com.cn/market/view/>
- [ChatGPT](https://chatgpt.com/share/68367a94-77a4-8005-8b42-d4862d345757)

In [ ]:
# 文件路径设定{tag}
# Note：进行后续分析之前，请先执行本 Cell 中的代码，以确保所有路径正确设置。

import os
import sys
import warnings
warnings.filterwarnings("ignore")

# 设置根目录
path = r'D:\Github\ds\cases\web_get_SZ_overall'
os.chdir(path)

# Folders 
data_folder = os.path.join(path, 'data')       # 原始数据存放目录
data_clean_folder = os.path.join(path, 'data_clean')  # 清洗后的数据存放目录

# 创建数据目录
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
if not os.path.exists(data_clean_folder):
    os.makedirs(data_clean_folder)

# 数据文件路径
os.chdir(data_folder)
print('Working Directory:', os.getcwd())

Working Directory: D:\Github\ds\cases\web_get_SZ_overall\data


In [26]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime, timedelta

# === 1. 配置参数 ===
# 设定为前一个有效交易日（假设昨天为有效交易日，可根据实际情况调整）
today = datetime.today()
date = (today - timedelta(days=1)).strftime("%Y-%m-%d")


# === 2. 构建请求参数 ===
url = (
    "https://query.sse.com.cn/commonExcelDd.do?"
    f"sqlId=COMMON_SSE_SJ_SCGM_C&isPagination=false&TRADE_DATE={date}"
)
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.sse.com.cn/market/view/",
}

# === 3. 发起请求 ===
response = requests.get(url, headers=headers)
content_type = response.headers.get('Content-Type', '')

# === 4. 判断格式并读取 ===
if response.status_code == 200:
    try:
        # 尝试读取 Excel 格式
        if 'application/vnd.ms-excel' in content_type or 'application/octet-stream' in content_type \
           or response.content[:2] == b'\xD0\xCF':  # xls
            df = pd.read_excel(BytesIO(response.content))
            print("数据以 Excel 格式返回")
        else:
            # 尝试读取 CSV 格式
            df = pd.read_csv(BytesIO(response.content), encoding="gbk")
            print("数据以 CSV 格式返回")

        # === 5. 保存数据 ===
        df.to_excel(f"sse_market_overview.xlsx", index=False)
        df.to_csv(f"sse_market_overview.csv", index=False, encoding="gbk")
        print(f"数据已保存为 sse_market_overview.xlsx 和 .csv")
    except Exception as e:
        print("读取数据文件时发生错误：", e)
else:
    print("下载失败，状态码：", response.status_code)

print("\n数据预览：\n")
print(df.head(30))

数据以 Excel 格式返回
数据已保存为 sse_market_overview.xlsx 和 .csv

数据预览：

             分类  数量(只)    总市值(亿元)   流通市值(亿元)   成交额(亿元)
0            股票   2323  525495.49  499102.27   3954.55
1           主板A   1693  458682.92  446821.09   3278.25
2           主板B     43     944.55     695.84      1.32
3           科创板    587   65868.02   51585.34    674.98
4           优先股     31    7599.43    7599.43      3.64
5            债券  32897  180125.91  180121.86   1104.75
6           政府债  12353   29915.06   29915.06    206.33
7           信用债  20544  150210.85   150206.8    898.43
8            基金    851   31239.85   30520.76   1502.74
9           ETF    689   29830.46   29830.46   1497.76
10          LOF    118      87.44      87.44      1.49
11  基础设施公募REITs     44    1321.95     602.86      3.48
12           期权      5          -          -     17.50
13           回购     45          -          -  23293.63
14           其他     45          -          -     24.50
